# Worksheet 08

Name: Mahdi Khemakhem  
UID: U18251472

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.5060304612198774, 3.5480213091720603, 4.39734035049243, 2.7555292543363454, 2.560373482520803, 5.552404894040864, 5.052286393242134, 4.6666275109442665, 6.7774627782583945, 4.956594177450369]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.682334343912792, 8.856808058048363, 7.522581633032604, 7.554469527623343, 7.1825055070047465, 6.332479367198279, 7.786583467254885, 8.440734674108501, 8.950408102674082, 9.078009528496661]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[9.078009528496661, 4.956594177450369, 6.7774627782583945, 4.6666275109442665, 8.950408102674082, 8.440734674108501, 7.786583467254885, 5.052286393242134, 6.332479367198279, 7.1825055070047465]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Random pick = 0.5

Mean and variance of the first distribution = 5, 1

Mean and variance of the second distribution = 8, 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.956594177450369, 6.7774627782583945, 4.6666275109442665, 5.052286393242134, 6.332479367198279]
[9.078009528496661, 8.950408102674082, 8.440734674108501, 7.786583467254885, 7.1825055070047465]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.557090045418688,  mean_2 = 8.287648255907776
var_1 = 0.6997767494777781,  var_2 = 0.5119526850024516


c:\Users\kmfoc\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


They are somewhat close to the true values of (5, 1) and (8, 1), but not exactly the same. Especially the variances are off

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [8]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.078009528496661
probability of observing that point if it came from cluster 0 =  0.0002337700681835683
probability of observing that point if it came from cluster 1 =  0.2764561323415666
point =  4.956594177450369
probability of observing that point if it came from cluster 0 =  0.3272655429078298
probability of observing that point if it came from cluster 1 =  2.9866792213561305e-09
point =  6.7774627782583945
probability of observing that point if it came from cluster 0 =  0.20443016437995826
probability of observing that point if it came from cluster 1 =  0.012654558748555946
point =  4.6666275109442665
probability of observing that point if it came from cluster 0 =  0.23670499864732034
probability of observing that point if it came from cluster 1 =  9.2430065389345e-11
point =  8.950408102674082
probability of observing that point if it came from cluster 0 =  0.0004077856009912885
probability of observing that point if it came from cluster 1 =  0.3741267220033838
point = 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [9]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([(x[0] - mean[0])**2 * x[1] for x in zip(data, prob_s0_x)]) / sum(prob_s0_x), sum([(x[0] - mean[1])**2 * x[1] for x in zip(data, prob_s1_x)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5271899397481883,  P(S_2) = 0.47281006025181177
mean_1 = 5.741750789183606,  mean_2 = 8.421365500417
var_1 = 0.9155775529968989,  var_2 = 0.42653524002310134


Variance of 1 is better but of 2 is worse, both means are worse, but the probabilities are better

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe

In [10]:
# Update P(S_j|X_i)
prob_s0_x, prob_s1_x = [], []
for p in data:
    pdf_i = [norm.pdf(p, mean[j], np.sqrt(var[j])) for j in range(k)]
    prob_x_i = sum([prob_s[j] * pdf_i[j] for j in range(k)])

    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x_i)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x_i)

prob_s = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([(x[0] - mean[0]) ** 2 * x[1] for x in zip(data, prob_s0_x)]) / sum(prob_s0_x),
       sum([(x[0] - mean[1]) ** 2 * x[1] for x in zip(data, prob_s1_x)]) / sum(prob_s1_x)]

print("Updated Values after 1 iteration:")
print("P(S_1) =", prob_s[0], ", P(S_2) =", prob_s[1])
print("mean_1 =", mean[0], ", mean_2 =", mean[1])
print("var_1 =", var[0], ", var_2 =", var[1])

Updated Values after 1 iteration:
P(S_1) = 0.5628422376186586 , P(S_2) = 0.43715776238134135
mean_1 = 5.763687804501074 , mean_2 = 8.414175673801259
var_1 = 0.9668244415347331 , var_2 = 0.4509898872265725


Mean2 is better than  the last iteration, but Mean1 is worse than the last iteration,both variances are slightly better. P(S1) and P(S2) are worse than the last iteration.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [11]:
labels = [0 if prob_s0 > prob_s1 else 1 for prob_s0, prob_s1 in zip(prob_s0_x, prob_s1_x)]

for i, label in enumerate(labels):
    print(f"Data point {data[i]:.2f} is assigned to cluster {label}")


Data point 9.08 is assigned to cluster 1
Data point 4.96 is assigned to cluster 0
Data point 6.78 is assigned to cluster 0
Data point 4.67 is assigned to cluster 0
Data point 8.95 is assigned to cluster 1
Data point 8.44 is assigned to cluster 1
Data point 7.79 is assigned to cluster 1
Data point 5.05 is assigned to cluster 0
Data point 6.33 is assigned to cluster 0
Data point 7.18 is assigned to cluster 0
